In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import time
import re
import json
from urllib.request import urlretrieve
import requests

In [2]:
# Define the required scopes
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Authorize the client
creds = ServiceAccountCredentials.from_json_keyfile_name('lego-441023-0d2c9d44cde8.json', scopes)
client = gspread.authorize(creds)

# Open the Google Sheet
spreadsheet = client.open('inwestycje') 

sheet = spreadsheet.worksheet('LEGO') 

In [3]:
# Get all records from the sheet (headers are automatically used)
data = sheet.get_all_records(head=1)

df = pd.DataFrame(data)
num_rows = df.shape[0]
print(f"Number of rows: {num_rows}")
df = df.iloc[:-1]  

print(df.head())  

Number of rows: 16
  lp                                              check nr zestawu  \
0  1  https://promoklocki.pl/lego-brickheadz-40466-p...      40466   
1  2  https://promoklocki.pl/lego-dc-super-heroes-76...      76238   
2  3  https://promoklocki.pl/lego-harry-potter-76393...      76393   
3  4  https://promoklocki.pl/lego-promocyjne-40512-z...      40512   
4  5  https://promoklocki.pl/lego-brickheadz-40623-b...      40623   

                                               nazwa  \
0                          Pandy na Chiński Nowy Rok   
1  Maska Batmana z klasycznego serialu telewizyjnego   
2                    Harry Potter i Hermiona Granger   
3               Zabawa i styl - zestaw dodatkowy VIP   
4                          Bohaterowie bitwy o Endor   

                             nazwa ang           gtin cena wejściowa  \
0              Chinese New Year Pandas  5702016910704          53.99   
1        Classic TV Series Batman Cowl  5702017072449         195.99   
2  Harr

In [4]:
def getJSON(url):
    print(f"URL passed to priceGen: {url}") 
    # Setup the WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    
    # Get the page source after JavaScript is executed
    page_content = driver.page_source
    
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Extract the title of the page
    title = soup.title.string
    print("Tytuł strony:", title)
    
    script_tag = soup.find('script', type='application/ld+json')
    
    if script_tag:
        # Parse the JSON content
        data = json.loads(script_tag.string)
    else:
        print("JSON data not found in <script> tag.")
        data = []
    driver.quit()
    data_dict = data[1]
    return data_dict

In [5]:
start_time = time.time() 
for index, row in df.iterrows():
    data_dict = getJSON(row['check'])
    # print(data_dict)
    match = re.search(r'-\s*(.*)', data_dict['name'])
    if match:
        df.loc[index, 'nazwa'] = match.group(1)
    else:
        df.loc[index, 'nazwa'] = data_dict['name']
    match = re.search(r'\((.*?)\)', data_dict['description'])
    if match:
        df.loc[index, 'nazwa ang'] = match.group(1)
    else: 
        df.loc[index, 'nazwa ang'] = df.loc[index, 'nazwa']
    df.loc[index, 'gtin'] = data_dict['gtin']
    df.loc[index, 'teraz'] = float(data_dict['offers']['lowPrice'])
print(df.head())
end_time = time.time()  # Record the end time

# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Time passed: {elapsed_time:.2f} seconds")

URL passed to priceGen: https://promoklocki.pl/lego-brickheadz-40466-pandy-na-chinski-nowy-rok-p21201
Tytuł strony: LEGO 40466 BrickHeadz Pandy na Chiński Nowy Rok - porównaj ceny - promoklocki.pl
URL passed to priceGen: https://promoklocki.pl/lego-dc-super-heroes-76238-maska-batmana-z-klasycznego-serialu-telewizyjnego-p21439
Tytuł strony: LEGO 76238 DC Super Heroes Maska Batmana z klasycznego serialu telewizyjnego - porównaj ceny - promoklocki.pl
URL passed to priceGen: https://promoklocki.pl/lego-harry-potter-76393-harry-potter-i-hermiona-granger-p21316
Tytuł strony: LEGO 76393 Harry Potter Harry Potter i Hermiona Granger - porównaj ceny - promoklocki.pl
URL passed to priceGen: https://promoklocki.pl/lego-promocyjne-40512-zabawa-i-styl-zestaw-dodatkowy-vip-p22004
Tytuł strony: LEGO 40512 Promocyjne Zabawa i styl - zestaw dodatkowy VIP - porównaj ceny - promoklocki.pl
URL passed to priceGen: https://promoklocki.pl/lego-brickheadz-40623-bohaterowie-bitwy-o-endor-p22295
Tytuł strony: LE

In [6]:
# IT REMOVES FUNCTIONS - WRITES STRINGS - DONT DO THAT
# values = df.values.tolist()

# # Update the sheet with the new data (this will overwrite the current content)
# # The first row is the header, so we write the header separately
# sheet.update('A1', [df.columns.values.tolist()] + values)

columns_to_update = ['nazwa', 'nazwa ang', 'gtin', 'teraz']
df_update = df[columns_to_update]
# Convert the DataFrame with only to-update cols to a list of lists (values only, without headers)
values_to_update = df_update.values.tolist()

# Get the column indexes from the header row (you can adjust this if needed)
header_row = sheet.row_values(1)  # Read the header row from Google Sheets
col_indexes = [header_row.index(col) + 1 for col in columns_to_update]  # +1 because gspread uses 1-based index

# Update the Google Sheet with the new values
for i, col_index in enumerate(col_indexes):
    # Update the corresponding column in Google Sheets
    range_to_update = f"{chr(64 + col_index)}2:{chr(64 + col_index)}{len(df_update) + 1}"
    sheet.update(
        range_to_update,
        [[row[i]] for row in values_to_update],
        value_input_option='USER_ENTERED'  # This ensures values are written with user-like interpretation
    )

print("Google Sheet updated successfully for specified columns.")

C:\Users\sitko\AppData\Local\Temp\ipykernel_28872\3983690384.py:21: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(


Google Sheet updated successfully for specified columns.


In [11]:
# df.loc[:, 'cena wejściowa'] = df['cena wejściowa'].apply(clean_price)
cashin = round((df['cena wejściowa'] * df['ilość sztuk']).sum(), 2)
cashout = round((df['teraz'] * df['ilość sztuk']).sum(), 2)
profit = round((cashout-cashin)/cashin*100, 2)

print(f"Cash in = {cashin}")
print(f"Cash out = {cashout}")
print(f"Profit = {profit}%")

Cash in = 5023.78
Cash out = 6399.5
Profit = 27.38%
